In [13]:
import pandas as pd
import urllib.request
import xmltodict


In [16]:
sequences_df = pd.DataFrame({'chr':['chr1','chr2'],'start':['1200000','1300000'],'end':['1201000','1301000']})
sequences_df

,chr,start,end
0,chr1,1200000,1201000
1,chr2,1300000,1301000


In [17]:
# add padding
pad_bp = 25
sequences_df['pad_start'] = sequences_df['start'].map(int) - pad_bp
sequences_df['pad_end'] = sequences_df['end'].map(int)  + pad_bp - 1
sequences_df

,chr,start,end,pad_start,pad_end
0,chr1,1200000,1201000,1199975,1201024
1,chr2,1300000,1301000,1299975,1301024


In [21]:
# a function to get the sequences that you want api

def get_sequence(row,chr_col='chr', start_col='start', stop_col='end', version='hg19'):
    url = 'http://genome.ucsc.edu/cgi-bin/das/{}/dna?segment={}:{},{}'.format(version, row[chr_col],row[start_col],row[stop_col])#chr4:39355986,39356120'
    contents = urllib.request.urlopen(url).read()
    sequence = ''.join(xmltodict.parse(contents)['DASDNA']['SEQUENCE']['DNA']['#text'].upper().split('\n'))
    return(sequence)


In [22]:
%%time
sequences_df['sequence'] = sequences_df.apply(lambda x: get_sequence(x, 
                                                                     version='hg38',
                                                                     chr_col='chr', 
                                                                     start_col='pad_start',
                                                                     stop_col='pad_end'),axis=1)


CPU times: user 3.95 ms, sys: 1.68 ms, total: 5.63 ms
Wall time: 147 ms


In [23]:
sequences_df

,chr,start,end,pad_start,pad_end,sequence
0,chr1,1200000,1201000,1199975,1201024,GTCTGCGAGGTCACAACAGCATGGACCACCAGGGAGAGGGGGACTG...
1,chr2,1300000,1301000,1299975,1301024,AGGGCACTTACCATCACTGCTCTGAAGGGCGTGCAAGACAGGTGCC...
